In [1]:
#%run B_equalize-histograms-laggui.ipynb
#%run B_equalize-histograms-mask.ipynb
%run B_equalize-histograms-sklearn.ipynb

684=


In [2]:
ttl.args

Namespace(tag='2021-02-17_TimeTeleScope', date='2021-02-17', in_path='../orchid', cache_path='cache_path', videos_path='videos', figwidth=15, ds=1, fps=12, verb=0)

In [3]:
fnames = ttl.glob()

In [5]:
frames = []
n_bits = 16

for i, fname_new in enumerate(fnames):
    print(f'{fname_new=}')
    
    image_matched = histMatch(ttl.process(fname_new), image_reference)
    #print(image_matched.min(), image_matched.max())
    image_matched *= 2**n_bits-1
    image_matched = image_matched.astype(np.uint16)

    fname = f'{ttl.args.cache_path}/{ttl.args.tag}_{i}.png'
    
    import cv2
    cv2.imwrite(fname, cv2.cvtColor(image_matched, cv2.COLOR_RGB2BGR))

    frames.append(fname)
    

fname_new='../orchid/2021-02-06.20:12:04.jpg'
fname_new='../orchid/2021-02-06.20:12:10.jpg'
fname_new='../orchid/2021-02-06.20:20:01.jpg'
fname_new='../orchid/2021-02-06.20:20:07.jpg'
fname_new='../orchid/2021-02-06.20:30:01.jpg'
fname_new='../orchid/2021-02-06.20:30:07.jpg'
fname_new='../orchid/2021-02-06.20:40:01.jpg'
fname_new='../orchid/2021-02-06.20:40:06.jpg'
fname_new='../orchid/2021-02-06.22:00:01.jpg'
fname_new='../orchid/2021-02-06.22:00:06.jpg'
fname_new='../orchid/2021-02-06.22:10:01.jpg'
fname_new='../orchid/2021-02-06.22:10:07.jpg'
fname_new='../orchid/2021-02-06.22:20:01.jpg'
fname_new='../orchid/2021-02-06.22:20:06.jpg'
fname_new='../orchid/2021-02-06.22:30:01.jpg'
fname_new='../orchid/2021-02-06.22:30:07.jpg'
fname_new='../orchid/2021-02-06.22:40:01.jpg'
fname_new='../orchid/2021-02-06.22:40:07.jpg'
fname_new='../orchid/2021-02-06.22:50:01.jpg'
fname_new='../orchid/2021-02-06.22:50:07.jpg'
fname_new='../orchid/2021-02-06.23:00:01.jpg'
fname_new='../orchid/2021-02-06.23

In [15]:
import moviepy.editor as mpy
import moviepy.video.io.ImageSequenceClip
def make_mp4(moviename, fnames, fps, do_delete=True):
    clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(fnames, fps=fps)
    clip.write_videofile(moviename)
    if do_delete: 
        for fname in fnames: os.remove(fname)
    return moviename

In [6]:
video_name = f'{ttl.args.videos_path}/{ttl.args.tag}.mp4'
make_mp4(video_name, frames, ttl.args.fps, do_delete=False)

t:   0%|          | 2/685 [00:00<00:51, 13.36it/s, now=None]

Moviepy - Building video videos/2021-02-17_TimeTeleScope.mp4.
Moviepy - Writing video videos/2021-02-17_TimeTeleScope.mp4



Moviepy - Done !
Moviepy - video ready videos/2021-02-17_TimeTeleScope.mp4


'videos/2021-02-17_TimeTeleScope.mp4'

In [15]:
#from IPython.display import Image, display
width = 1024

def show(filename, width=1024):
    from IPython.display import Video
    # https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html?highlight=IPython.display#IPython.display.Video
    return Video(filename, html_attributes=f"controls muted autoplay width={width}")

In [8]:
show(video_name, width=1024)

In [9]:
if verb:

    image_matched = histMatch(ttl.process(image_new), ttl.process(image_reference))
    fig, ax = plt.subplots(figsize=ttl.figsize)
    ax.imshow(image_matched, vmin=0, vmax=255);

    image_matched.min(), image_matched.max()

In [19]:
def make_gif(gifname, fnames, fps, ds=1, tds=1, do_delete=True):
    import imageio
    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for fname in fnames[::tds]:
            img = imageio.imread(fname)
            if ds > 1:
                nx, ny, three = img.shape
                # CAREFUL ! for a strange reason openCV inverts H and W ??
                img = cv2.resize(img, (ny//ds, nx//ds),
                                interpolation = cv2.INTER_AREA)
                nx, ny, three = img.shape            
            writer.append_data(img)

    from pygifsicle import optimize
    optimize(str(gifname))
    if do_delete: 
        for fname in fnames: os.remove(fname)
    return gifname

In [20]:
gif_name = f'{ttl.args.videos_path}/{ttl.args.tag}.gif'
make_gif(gif_name, frames, ttl.args.fps, ds=8, tds=4, do_delete=False)

'videos/2021-02-17_TimeTeleScope.gif'